In [33]:
 !pip install MySQL-connector-python

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Q1: Some of the facilities charge a fee to members, but some do not.
Please list the names of the facilities that do.

In [8]:
from passwords import password, database
import mysql.connector
db = mysql.connector.connect(password=password, database=database, user='root',
                              host='localhost'
                              )

cur = db.cursor()


cur.execute('''SELECT name
FROM  `Facilities` 
WHERE membercost != 0.0
LIMIT 0 , 30''')

for i in cur:
    for j in i:
        print(j)
       


Tennis Court 1
Tennis Court 2
Massage Room 1
Massage Room 2
Squash Court


Q2: How many facilities do not charge a fee to members?

In [2]:
cur.execute('''SELECT COUNT( name ) 
FROM  `Facilities` 
WHERE membercost = 0.0''')

for i in cur:
    for j in i:
        print(j)

4


Q3: How can you produce a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost?
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. 

In [25]:

cur.execute('''SELECT facid, name, membercost, monthlymaintenance
    FROM  `Facilities` 
    WHERE membercost < ( 0.2 * monthlymaintenance ) 
    LIMIT 0 , 30''')

field_names = [i[0] for i in cur.description]
print(field_names)
for i in cur:
    print(i)

['facid', 'name', 'membercost', 'monthlymaintenance']
(0, 'Tennis Court 1', Decimal('5.0'), 200)
(1, 'Tennis Court 2', Decimal('5.0'), 200)
(2, 'Badminton Court', Decimal('0.0'), 50)
(3, 'Table Tennis', Decimal('0.0'), 10)
(4, 'Massage Room 1', Decimal('9.9'), 3000)
(5, 'Massage Room 2', Decimal('9.9'), 3000)
(6, 'Squash Court', Decimal('3.5'), 80)
(7, 'Snooker Table', Decimal('0.0'), 15)
(8, 'Pool Table', Decimal('0.0'), 15)


Q4: How can you retrieve the details of facilities with ID 1 and 5?
Write the query without using the OR operator.

In [26]:
cur.execute('''SELECT * 
FROM  `Facilities` 
WHERE facid
IN ( 1, 5 ) 
LIMIT 0 , 30''')

field_names = [i[0] for i in cur.description]
print(field_names)
for i in cur:
    print(i)

['facid', 'name', 'membercost', 'guestcost', 'initialoutlay', 'monthlymaintenance']
(1, 'Tennis Court 2', Decimal('5.0'), Decimal('25.0'), 8000, 200)
(5, 'Massage Room 2', Decimal('9.9'), Decimal('80.0'), 4000, 3000)


Q5: How can you produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100? Return the name and monthly maintenance of the facilities
in question. 

In [27]:
cur.execute('''SELECT name, monthlymaintenance, 
CASE WHEN monthlymaintenance >100
THEN  'expensive'
ELSE  'cheap'
END AS label
FROM  `Facilities` 
WHERE 1 
LIMIT 0 , 30''')

field_names = [i[0] for i in cur.description]
print(field_names)
for i in cur:
    print(i)

['name', 'monthlymaintenance', 'label']
('Tennis Court 1', 200, 'expensive')
('Tennis Court 2', 200, 'expensive')
('Badminton Court', 50, 'cheap')
('Table Tennis', 10, 'cheap')
('Massage Room 1', 3000, 'expensive')
('Massage Room 2', 3000, 'expensive')
('Squash Court', 80, 'cheap')
('Snooker Table', 15, 'cheap')
('Pool Table', 15, 'cheap')


Q6: You'd like to get the first and last name of the last member(s)
who signed up. Do not use the LIMIT clause for your solution.

In [28]:
cur.execute('''SELECT firstname, surname, joindate
FROM  `Members` 
WHERE joindate = ( 
SELECT MAX( joindate ) 
FROM  `Members` )''')

field_names = [i[0] for i in cur.description]
print(field_names)
for i in cur:
    print(i)

['firstname', 'surname', 'joindate']
('Darren', 'Smith', '2012-09-26 18:08:45')


Q7: How can you produce a list of all members who have used a tennis court?
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

In [29]:
cur.execute('''SELECT DISTINCT (
CONCAT( firstname,  ' ', surname )
) AS name, 
CASE WHEN facid =0
THEN  'Tennis Court 1'
WHEN facid =1
THEN  'Tennis Court 2'
END AS court_name
FROM  `Members` member
INNER JOIN  `Bookings` booking ON booking.memid = member.memid
WHERE facid
IN ( 0, 1 ) 
ORDER BY name''')

field_names = [i[0] for i in cur.description]
print(field_names)
for i in cur:
    print(i)

['name', 'court_name']
('Anne Baker', 'Tennis Court 1')
('Anne Baker', 'Tennis Court 2')
('Burton Tracy', 'Tennis Court 2')
('Burton Tracy', 'Tennis Court 1')
('Charles Owen', 'Tennis Court 1')
('Charles Owen', 'Tennis Court 2')
('Darren Smith', 'Tennis Court 2')
('David Farrell', 'Tennis Court 2')
('David Farrell', 'Tennis Court 1')
('David Jones', 'Tennis Court 1')
('David Jones', 'Tennis Court 2')
('David Pinker', 'Tennis Court 1')
('Douglas Jones', 'Tennis Court 1')
('Erica Crumpet', 'Tennis Court 1')
('Florence Bader', 'Tennis Court 1')
('Florence Bader', 'Tennis Court 2')
('Gerald Butters', 'Tennis Court 2')
('Gerald Butters', 'Tennis Court 1')
('GUEST GUEST', 'Tennis Court 2')
('GUEST GUEST', 'Tennis Court 1')
('Henrietta Rumney', 'Tennis Court 2')
('Jack Smith', 'Tennis Court 2')
('Jack Smith', 'Tennis Court 1')
('Janice Joplette', 'Tennis Court 2')
('Janice Joplette', 'Tennis Court 1')
('Jemima Farrell', 'Tennis Court 1')
('Jemima Farrell', 'Tennis Court 2')
('Joan Coplin', 'T

Q8: How can you produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30? Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

In [30]:
cur.execute('''SELECT CONCAT( firstname,  ' ', surname ) AS full_name, name, 
CASE WHEN firstname =  'GUEST'
THEN guestcost * slots
ELSE membercost * slots
END AS total_cost
FROM  `Bookings` booking
INNER JOIN  `Facilities` facility ON booking.facid = facility.facid
INNER JOIN  `Members` member ON booking.memid = member.memid
WHERE (
starttime LIKE  '%2012-09-14%'
)
AND (
(
firstname =  'GUEST'
AND guestcost * slots >30
)
OR (
firstname !=  'GUEST'
AND membercost * slots >30
)
)
ORDER BY total_cost DESC''')

field_names = [i[0] for i in cur.description]
print(field_names)
for i in cur:
    print(i)

['full_name', 'name', 'total_cost']
('GUEST GUEST', 'Massage Room 2', Decimal('320.0'))
('GUEST GUEST', 'Massage Room 1', Decimal('160.0'))
('GUEST GUEST', 'Massage Room 1', Decimal('160.0'))
('GUEST GUEST', 'Massage Room 1', Decimal('160.0'))
('GUEST GUEST', 'Tennis Court 2', Decimal('150.0'))
('GUEST GUEST', 'Tennis Court 1', Decimal('75.0'))
('GUEST GUEST', 'Tennis Court 1', Decimal('75.0'))
('GUEST GUEST', 'Tennis Court 2', Decimal('75.0'))
('GUEST GUEST', 'Squash Court', Decimal('70.0'))
('Jemima Farrell', 'Massage Room 1', Decimal('39.6'))
('GUEST GUEST', 'Squash Court', Decimal('35.0'))
('GUEST GUEST', 'Squash Court', Decimal('35.0'))


Q9: This time, produce the same result as in Q8, but using a subquery.

In [31]:
cur.execute('''SELECT * 
FROM (

SELECT CONCAT( firstname,  ' ', surname ) AS full_name, name, 
CASE WHEN firstname =  'GUEST'
THEN guestcost * slots
ELSE membercost * slots
END AS total_cost
FROM  `Bookings` booking
INNER JOIN  `Facilities` facility ON booking.facid = facility.facid
INNER JOIN  `Members` member ON booking.memid = member.memid
WHERE (
starttime
IN (

SELECT starttime
FROM  `Bookings` 
WHERE starttime LIKE  '%2012-09-14%'
)
)
AND (
(
firstname
IN (

SELECT firstname
FROM  `Members` 
WHERE firstname =  'GUEST'
AND guestcost * slots >30
)
)
OR (
firstname
IN (

SELECT firstname
FROM  `Members` 
WHERE firstname !=  'GUEST'
AND membercost * slots >30
)
)
)
ORDER BY total_cost DESC
)t''')
print(field_names)
for i in cur:
    print(i)

['full_name', 'name', 'total_cost']
('GUEST GUEST', 'Massage Room 2', Decimal('320.0'))
('GUEST GUEST', 'Massage Room 1', Decimal('160.0'))
('GUEST GUEST', 'Massage Room 1', Decimal('160.0'))
('GUEST GUEST', 'Massage Room 1', Decimal('160.0'))
('GUEST GUEST', 'Tennis Court 2', Decimal('150.0'))
('GUEST GUEST', 'Tennis Court 1', Decimal('75.0'))
('GUEST GUEST', 'Tennis Court 1', Decimal('75.0'))
('GUEST GUEST', 'Tennis Court 2', Decimal('75.0'))
('GUEST GUEST', 'Squash Court', Decimal('70.0'))
('Jemima Farrell', 'Massage Room 1', Decimal('39.6'))
('GUEST GUEST', 'Squash Court', Decimal('35.0'))
('GUEST GUEST', 'Squash Court', Decimal('35.0'))


Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [32]:
cur.execute('''SELECT * 
FROM (

SELECT name, SUM( 
CASE WHEN memid !=0
THEN membercost * slots
WHEN memid =0
THEN guestcost * slots
END ) AS revenue
FROM  `Facilities` facility
JOIN  `Bookings` booking ON facility.facid = booking.facid
GROUP BY name
ORDER BY revenue DESC
)t
WHERE revenue <1000''')

print(field_names)
for i in cur:
    print(i)


['full_name', 'name', 'total_cost']
('Pool Table', Decimal('270.0'))
('Snooker Table', Decimal('240.0'))
('Table Tennis', Decimal('180.0'))
